## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
detail_results = pd.merge(mouse_metadata, study_results, on="Mouse ID", how="right")

# Display the data table for preview
detail_results

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,Capomulin,Female,9,22,0,45.000000,0
1,b128,Capomulin,Female,9,22,5,45.651331,0
2,b128,Capomulin,Female,9,22,10,43.270852,0
3,b128,Capomulin,Female,9,22,15,43.784893,0
4,b128,Capomulin,Female,9,22,20,42.731552,0
...,...,...,...,...,...,...,...,...
1888,m601,Capomulin,Male,22,17,25,33.118756,1
1889,m601,Capomulin,Male,22,17,30,31.758275,1
1890,m601,Capomulin,Male,22,17,35,30.834357,1
1891,m601,Capomulin,Male,22,17,40,31.378045,1


In [2]:
# Checking the number of mice.
num_mice = len(detail_results["Mouse ID"].unique())
num_mice

249

In [9]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
count_mice_time = pd.DataFrame(detail_results[["Mouse ID","Timepoint"]].value_counts()).reset_index()
duplicate_mice_time = count_mice_time.loc[count_mice_time[0] > 1, :]
duplicate_mice_ids = duplicate_mice_time["Mouse ID"].unique()
duplicate_mice_ids

array(['g989'], dtype=object)

In [10]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_mice_data = detail_results.loc[detail_results["Mouse ID"] == duplicate_mice_ids[0], :]
duplicate_mice_data

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
860,g989,Propriva,Female,21,26,0,45.000000,0
861,g989,Propriva,Female,21,26,0,45.000000,0
862,g989,Propriva,Female,21,26,5,48.786801,0
863,g989,Propriva,Female,21,26,5,47.570392,0
864,g989,Propriva,Female,21,26,10,51.745156,0
865,g989,Propriva,Female,21,26,10,49.880528,0
866,g989,Propriva,Female,21,26,15,51.325852,1
867,g989,Propriva,Female,21,26,15,53.442020,0
868,g989,Propriva,Female,21,26,20,55.326122,1
869,g989,Propriva,Female,21,26,20,54.657650,1


In [11]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_results = detail_results.loc[detail_results["Mouse ID"] != duplicate_mice_ids[0], :]
clean_results

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,Capomulin,Female,9,22,0,45.000000,0
1,b128,Capomulin,Female,9,22,5,45.651331,0
2,b128,Capomulin,Female,9,22,10,43.270852,0
3,b128,Capomulin,Female,9,22,15,43.784893,0
4,b128,Capomulin,Female,9,22,20,42.731552,0
...,...,...,...,...,...,...,...,...
1888,m601,Capomulin,Male,22,17,25,33.118756,1
1889,m601,Capomulin,Male,22,17,30,31.758275,1
1890,m601,Capomulin,Male,22,17,35,30.834357,1
1891,m601,Capomulin,Male,22,17,40,31.378045,1


In [12]:
# Checking the number of mice in the clean DataFrame.
num_mice = len(clean_results["Mouse ID"].unique())
num_mice

248

## Summary Statistics

In [48]:
from IPython.core.display import HTML
HTML("<style>.rendered_html th {max-width: 100px;}</style>")

In [49]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

regimen_group = clean_results.groupby("Drug Regimen")
summary_stats_df = pd.DataFrame(regimen_group["Drug Regimen"].unique())
summary_stats_df["Mean Tumor Volume"] = regimen_group["Tumor Volume (mm3)"].mean()
summary_stats_df["Median Tumor Volume"] = regimen_group["Tumor Volume (mm3)"].median()
summary_stats_df["Variance Tumor Volume"] = regimen_group["Tumor Volume (mm3)"].var()
summary_stats_df["Standard Deviation Tumor Volume"] = regimen_group["Tumor Volume (mm3)"].std()

del summary_stats_df["Drug Regimen"]

for x in range(0,4):
    summary_stats_df.iloc[:,x] = summary_stats_df.iloc[:,x].map("{:.2f}".format)
    
summary_stats_df

,Mean Tumor Volume,Median Tumor Volume,Variance Tumor Volume,Standard Deviation Tumor Volume
Drug Regimen,,,,
Capomulin,40.68,41.56,24.95,4.99
Ceftamin,52.59,51.78,39.29,6.27
Infubinol,52.88,51.82,43.13,6.57
Ketapril,55.24,53.70,68.55,8.28
Naftisol,54.33,52.51,66.17,8.13
Placebo,54.03,52.29,61.17,7.82
Propriva,52.32,50.45,43.85,6.62
Ramicane,40.22,40.67,23.49,4.85
Stelasyn,54.23,52.43,59.45,7.71


In [50]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# Using the aggregation method, produce the same summary statistics in a single line
simple_summary_stats_df = regimen_group["Tumor Volume (mm3)"].agg(["mean","median","var","std"]) \
                        .rename(columns={"mean": "Mean Tumor Volume", 
                                         "median": "Median Tumor Volume", 
                                         "var": "Variance Tumor Volume", 
                                         "std": "Standard Deviation Tumor Volume"})
for x in range(0,4):
    simple_summary_stats_df.iloc[:,x] = simple_summary_stats_df.iloc[:,x].map("{:.2f}".format)

simple_summary_stats_df

,Mean Tumor Volume,Median Tumor Volume,Variance Tumor Volume,Standard Deviation Tumor Volume
Drug Regimen,,,,
Capomulin,40.68,41.56,24.95,4.99
Ceftamin,52.59,51.78,39.29,6.27
Infubinol,52.88,51.82,43.13,6.57
Ketapril,55.24,53.70,68.55,8.28
Naftisol,54.33,52.51,66.17,8.13
Placebo,54.03,52.29,61.17,7.82
Propriva,52.32,50.45,43.85,6.62
Ramicane,40.22,40.67,23.49,4.85
Stelasyn,54.23,52.43,59.45,7.71


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.



In [10]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
